<a href="https://colab.research.google.com/github/anlianguzova/BI-stat-course-2022/blob/hw_DEG_cont/python_code/hw_deg_cont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as st
from statsmodels.stats.weightstats import ztest


## Code from previous homework

In [ ]:
data_path = "data"
expression_data = pd.read_csv(f"{data_path}/homework_lecture_5_data.csv", index_col=0)
expression_data.head()

In [ ]:
b_cells_expression_data = expression_data.query("Cell_type == 'B_cell'")
nk_cells_expression_data = expression_data.query("Cell_type == 'NK_cell'")

example_gene = "TMCC1"

sns.histplot(b_cells_expression_data[example_gene], stat="density")

In [ ]:
sns.histplot(nk_cells_expression_data[example_gene], stat="density")

In [ ]:
def demonstrate_clt(expressions, sample_size: int = 1000, n_samples: int = 1000):
    randoms = np.array([
        np.random.choice(
            expressions, size=sample_size
        ) for _ in range(n_samples)
    ])
    mean_expressions = randoms.mean()
    return mean_expressions

In [ ]:
b_example = b_cells_expression_data[example_gene]
nk_example = nk_cells_expression_data[example_gene]

b_dem_res = demonstrate_clt(b_example)
nk_dem_res = demonstrate_clt(nk_example)
b_dem_res, nk_dem_res

In [ ]:
# Confidence interval for B-cells
def calculate_ci(sample, mean_mean, std = None):
    if std is None:
      std = sample.std()

    se = std / np.sqrt(len(sample))
    left_b = mean_mean - 1.96 * se
    right_b = mean_mean + 1.96 * se

    return left_b, right_b

b_int = calculate_ci(b_example, b_dem_res)

b_int, b_dem_res

In [ ]:
# Confidence interval for NK-cells
nk_int = calculate_ci(nk_example, nk_dem_res)

nk_int, nk_dem_res

In [ ]:
st.t.interval(confidence = 0.95, 
              df = len(b_cells_expression_data[example_gene]) - 1, 
              loc = np.mean(b_cells_expression_data[example_gene]), 
              scale = st.sem(b_cells_expression_data[example_gene])) 

In [ ]:
st.t.interval(confidence = 0.95, 
              df = len(nk_cells_expression_data[example_gene]) - 1,
              loc = np.mean(nk_cells_expression_data[example_gene]),
              scale = st.sem(nk_cells_expression_data[example_gene]))

In [ ]:
def check_intervals_intersect(first_ci: tuple, second_ci: tuple):
    are_intersect = False
    if (first_ci[1] > second_ci[0]) and (first_ci[0] < second_ci[1]):
        are_intersect = True
    return are_intersect

In [ ]:
def get_names(table):
    return table.columns[:-1].values

gene_names_exp = get_names(expression_data)

In [ ]:
def interval_95(table, length):
    ans = st.t.interval(confidence=0.95,
                        df=length - 1,
                        loc=np.mean(table),
                        scale=st.sem(table))
    return ans

def interval_to_tables(first_table, second_table):
    first_intervals = []
    second_intervals = []
    gene_names = get_names(first_table)
    for gene in gene_names:
        df_f_length = len(first_table[gene])
        first_int = interval_95(first_table[gene], df_f_length)
        first_intervals.append(first_int)
        df_s_length = len(second_table[gene])
        second_int = interval_95(second_table[gene], df_s_length)
        second_intervals.append(second_int)
    return first_intervals, second_intervals

def check_dge_with_ci(first_table, second_table):
    f_intervals, s_intervals = interval_to_tables(first_table, second_table)
    ci_test_results = []
    for f_int, s_int in zip(
        f_intervals, s_intervals
    ):
        ci_test_results.append(check_intervals_intersect(f_int, s_int))
    return ci_test_results

check_dge_with_ci(b_cells_expression_data, nk_cells_expression_data)

## Multiple comparisons

In [ ]:
def corr_mult_comp(p_values: list, method: str) -> list:
    from statsmodels.stats.multitest import multipletests as mpt
    return list(mpt(p_values, method=method)[1])

def find_p_val(first_table, second_table):
    gene_names = get_names(first_table)
    z_stat_val = []
    for gene in gene_names:
        ans = ztest(
            first_table[gene],
            second_table[gene]
        )
        z_stat_val.append(ans)
    z_test_p_values = []
    for num in z_stat_val:
        z_test_p_values.append(num[1])
    return z_test_p_values

def test_p_val(p_values):
    z_test_results = []
    for val in p_values:
        if val < 0.05:
            z_test_results.append(True)
        else:
            z_test_results.append(False)
    return z_test_results


def check_dge_with_ztest(first_table, second_table, results=True, values=False, method=None):
    z_test_p_values = find_p_val(first_table, second_table)
    z_test_results = test_p_val(z_test_p_values)
    if method:
        cor_p_values = corr_mult_comp(z_test_p_values, method=method)
        z_test_cor_res = test_p_val(cor_p_values)
        if results:
            return z_test_cor_res
        if values:
            return cor_p_values
    else:
        if results:
            return z_test_results
        if values:
            return z_test_p_values

check_dge_with_ztest(b_cells_expression_data, nk_cells_expression_data, )

In [ ]:
def create_table(first_table, second_table, method=None):
    ci_test_results = check_dge_with_ci(first_table, second_table)
    z_test_results = check_dge_with_ztest(first_table, second_table, results=True)
    z_test_p_values = check_dge_with_ztest(first_table, second_table, results=False, values=True)
    #mean_diff = mean_differ(first_table, second_table)
    results = {
        "ci_test_results": ci_test_results,
        "z_test_results": z_test_results,
        "z_test_p_values": z_test_p_values,
        #"mean_diff": mean_diff
    }
    if method:
        z_test_cor_res = check_dge_with_ztest(first_table, second_table, results=True, method=method)
        z_test_cor_p_val = check_dge_with_ztest(first_table, second_table, results=False, values=True, method=method)
        results["z_test_cor_res"] = z_test_cor_res
        results["z_test_cor_p_values"] = z_test_cor_p_val
        return pd.DataFrame(results)
    print(pd.DataFrame(results))

create_table(b_cells_expression_data, nk_cells_expression_data, method="holm")